In [1]:
import pandas as pd
import scipy
from scipy.spatial.distance import pdist, squareform

df_concat = pd.read_csv("donnees_propres.csv")

In [2]:
df_concat.set_index('nom', inplace=True)
df_concat.head()

,women,men,hiver,printemps,ete,autone,jour,nuit,marine,savory,...,amber,nutty,musky,aldehydic,leather,terpenic,aquatic,balsamic,floral,yellow floral
nom,,,,,,,,,,,,,,,,,,,,,
A Nocturnal Whisper Perfume Oil Gucci for women and men,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0,0,...,1,0,0,0,1,0,0,0,0,0
A Reason To Love Gucci for women and men,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0,0,...,1,0,0,0,0,0,0,0,1,0
A Song For The Rose Eau de Parfum Gucci for women and men,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,...,0,0,1,0,0,0,0,0,1,0
A Winter Melody Scented Water Gucci for women and men,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bloom Gocce di Fiori Gucci for women,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
def prediction(*parfums):
    global df_concat
    new_user = np.zeros(df_concat.shape[1])  # Initialisation d'une série de zéros

    # Calcul du nouveau profil utilisateur
    for parfum in parfums:
        new_user += df_concat.loc[parfum]
    new_user[new_user > 1] = 1  # Limite à 1
    df_concat.loc['new_user'] = new_user
    
    # Calcul de la similarité de Jaccard
    jaccard_distances = pdist(df_concat.values, metric='jaccard')
    jaccard_similarity_array = 1 - squareform(jaccard_distances)
    jaccard_similarity_df = pd.DataFrame(jaccard_similarity_array, index=df_concat.index, columns=df_concat.index)
    
    # Récupération des similarités pour le nouvel utilisateur
    jaccard_similarity_series = jaccard_similarity_df.loc['new_user']
    
    # Suppression des parfums d'entrée
    jaccard_similarity_series = jaccard_similarity_series.drop(list(parfums))
    
    # Trier les valeurs de similarité de la plus élevée à la plus basse
    ordered_similarities = jaccard_similarity_series.sort_values(ascending=False)
    
    # Supprimer le profil utilisateur après la prédiction
    df_concat = df_concat.drop(index='new_user')
    
    L = ordered_similarities[1:6]
    dic = { L.index[0]: L.values[0],L.index[1]: L.values[1],L.index[2]: L.values[2],L.index[3]: L.values[3],
           L.index[4]: L.values[4]}
        
    return dic